### This notebook uses MTEB for embedding model comparisions
#### Author: Madhuroopa

In [13]:
%pip install --upgrade pinecone-client 

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/201.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/201.4 kB ? eta -:--:--
   ------- ------------------------------- 41.0/201.4 kB 991.0 kB/s eta 0:00:01
   -------------------------------------- - 194.6/201.4 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 201.4/201.4 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 2.2.4
    Uninstalling pinecone-client-2.2.4:
      Successfully uninstalled pinecone-client-2.2.4
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install cohere

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/52.1 kB ? eta -:--:--
   ---------------------------------------  51.2/52.1 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 52.1/52.1 kB 910.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/499.3 kB ? eta -:--:--
   ----------- ---------------------------- 143.4/499.3 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 499.3/499.3 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import dotenv
dotenv.load_dotenv()

True

In [16]:
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

#### Reading data 

In [17]:
transcript = pd.read_csv('Monthly_Release_Kickoff_1.csv')
transcript.head()

,Unnamed: 0,start_time,end_time,speaker_label,text
0,0,0.053500,0.079167,spk_0,"All right. Hi, everybody."
1,1,0.079483,0.141483,spk_0,Welcome to the Glab 16.7 monthly release kicko...
2,2,0.145333,0.174500,spk_0,I'm Mike Float and I'm gonna be your host today.
3,3,0.178983,0.214667,spk_0,Uh This is my first time hosting a release kic...
4,4,0.214833,0.250650,spk_0,Git Lab and I couldn't be more excited to be h...


In [18]:
transcript.dropna(inplace=True)
transcript.drop(['Unnamed: 0', 'end_time' ], axis=1, inplace=True)
transcript

,start_time,speaker_label,text
0,0.053500,spk_0,"All right. Hi, everybody."
1,0.079483,spk_0,Welcome to the Glab 16.7 monthly release kicko...
2,0.145333,spk_0,I'm Mike Float and I'm gonna be your host today.
3,0.178983,spk_0,Uh This is my first time hosting a release kic...
4,0.214833,spk_0,Git Lab and I couldn't be more excited to be h...
...,...,...,...
458,22.891483,spk_0,So um so much great stuff.
459,22.929500,spk_0,"I know we, we may have said this before,"
460,22.957483,spk_0,but I can't help but think based on everything.
461,22.997483,spk_0,Uh I've just heard this could be our best rele...


In [19]:
transcript['combined_text'] = ""

# Combine text in every two consecutive rows using a for loop
for i in range(0, len(transcript), 2):
    combined_text = " ".join(transcript.loc[i:i+2, 'text'])
    transcript.at[i, 'combined_text'] = combined_text

# Keep only rows where combined_text is not empty
transcript = transcript[transcript['combined_text'] != ""].reset_index(drop=True)


In [20]:
transcript

,start_time,speaker_label,text,combined_text
0,0.053500,spk_0,"All right. Hi, everybody.","All right. Hi, everybody. Welcome to the Glab ..."
1,0.145333,spk_0,I'm Mike Float and I'm gonna be your host today.,I'm Mike Float and I'm gonna be your host toda...
2,0.214833,spk_0,Git Lab and I couldn't be more excited to be h...,Git Lab and I couldn't be more excited to be h...
3,0.335983,spk_0,"So you're gonna hear from Vi Chetta, who's gon...","So you're gonna hear from Vi Chetta, who's gon..."
4,0.426667,spk_0,talk about some of the improvements coming in ...,talk about some of the improvements coming in ...
...,...,...,...,...
227,22.671333,spk_0,"Um code search, fantastic uh progress there.","Um code search, fantastic uh progress there. U..."
228,22.767817,spk_0,and um bringing uh capabilities to our,and um bringing uh capabilities to our uh self...
229,22.891483,spk_0,So um so much great stuff.,"So um so much great stuff. I know we, we may h..."
230,22.957483,spk_0,but I can't help but think based on everything.,but I can't help but think based on everything...


In [169]:
# Combining speakers's rows
import warnings
warnings.filterwarnings("ignore")
to_remove=[]
for i, record in transcript.iterrows():
    if i < len(transcript)-1:
        if transcript.speaker_label[i]== transcript.speaker_label[i+1]:
            transcript['text'][i]=transcript['text'][i]+" "+transcript['text'][i+1]
            to_remove.append(i+1)

transcript = transcript.drop(to_remove).reset_index(drop=True)

In [21]:
transcript.head()

,start_time,speaker_label,text,combined_text
0,0.053500,spk_0,"All right. Hi, everybody.","All right. Hi, everybody. Welcome to the Glab ..."
1,0.145333,spk_0,I'm Mike Float and I'm gonna be your host today.,I'm Mike Float and I'm gonna be your host toda...
2,0.214833,spk_0,Git Lab and I couldn't be more excited to be h...,Git Lab and I couldn't be more excited to be h...
3,0.335983,spk_0,"So you're gonna hear from Vi Chetta, who's gon...","So you're gonna hear from Vi Chetta, who's gon..."
4,0.426667,spk_0,talk about some of the improvements coming in ...,talk about some of the improvements coming in ...


In [84]:
next(iter(transcript))

'start_time'

In [22]:
import cohere
COHERE_API_KEY=os.getenv('COHERE_API_KEY')or 'COHERE_API_KEY'
co = cohere.Client('XuQPwqOnk7dNpkK4p36PKM79jQ8i4iPcv2BzMifH')

In [23]:
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

#### Embeding Models

##### Cohere Model:embed-english-v3.0  dimensions: 1024

In [24]:

def embed_text(texts):
  output = co.embed(
                model="embed-english-v3.0",
    						input_type="search_document",
                texts=texts)
  embedding = output.embeddings

  return embedding


#### Open AI Embedding Model
##### Model: text-embedding-ada-002 dimensions: 1536

In [25]:
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

C:\Users\madhu\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


### Pinecone

In [27]:
from pinecone import Pinecone, ServerlessSpec
import time

PINECONE_API_KEY = os.getenv('PINECONE_SERVERLESS_API_KEY') or 'PINECONE_SERVERLESS_API_KEY'
openai_index_name = 'openaiembeddings'
openai_namespace = 'monthlyreleasekickoff'
cohere_index_name = 'langchain-retrieval-transcript'
cohere_namespace = 'monthlyreleasekickoff'

pinecone = Pinecone(api_key=PINECONE_API_KEY)

In [29]:
if openai_index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=openai_index_name,
        metric='cosine',
        dimension=1536,  # model_name = 'text-embedding-ada-002'; 1536 dim of text-embedding-ada-002
        
        spec=ServerlessSpec(
        cloud='aws', 
        region='us-west-2'
        # pod_type="p1.x1",
        ) 
    )   
    
while not pinecone.describe_index(openai_index_name).status['ready']:
    time.sleep(1)


In [30]:
index = pinecone.Index(openai_index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [127]:
#index.delete(delete_all=True, namespace=namespace)

{}

#### Inserting data into pinecone

In [32]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 90
texts = []
metadatas = []
meeting_id = 1
start_id = 0

for i, record in tqdm(transcript.iterrows()):
    # first get metadata fields for this record
    metadata = {
        'speaker': record['speaker_label'],
        'start_time': round(record['start_time'], 4), # limit to 4 decimal places 
        'meeting_id': meeting_id,
        'text': record['combined_text'], # Storing the text in the metadata for now, later we'd need to decode it from vectors
    }

    record_texts = record['combined_text']

    texts.append(record_texts)
    metadatas.append(metadata)

    # print(texts)
    # print(metadatas)

    # if we've reached the batch limit, then index the batch
    if len(texts) >= batch_limit:
        #ids = [str(uuid4()) for _ in range(len(texts))]
        ids = [str(i+1) for i in range(start_id,(start_id + len(texts)))]
        start_id += len(texts)
        #embeds = embed_text(texts)
        embeds =embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas), namespace=openai_namespace)
        texts = []
        metadatas = []
        meeting_id += 1
# add any remaining texts to the index
if len(texts) > 0:
    #ids = [str(uuid4()) for _ in range(len(texts))]
    ids = [str(i+1) for i in range(start_id,(start_id + len(texts)))]
    #embeds = embed_text(texts)
    embeds =embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))
    
time.sleep(5)    

232it [00:21, 10.92it/s]


In [33]:
index.describe_index_stats()   

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 52},
                'monthlyreleasekickoff': {'vector_count': 180}},
 'total_vector_count': 232}

#### Cohere

In [136]:
query  = "What is the significance of GLAB pages supporting multiple versions, as mentioned in the meeting"
downstr_response = index.query(
    vector= embed_text([query])[0],
    # filter={
    #     "meeting_id": {"$in":[1, 2]}
    # },
    namespace=namespace, 
    top_k=20,
    include_metadata=True,
)
downstr_response

{'matches': [{'id': '48',
              'metadata': {'meeting_id': 1.0,
                           'speaker': 'spk_1',
                           'start_time': 2.2963,
                           'text': 'Next up, I want to discuss um get lab '
                                   'pages, multiple version support.'},
              'score': 0.622147799,
              'values': []},
             {'id': '408',
              'metadata': {'meeting_id': 5.0,
                           'speaker': 'spk_5',
                           'start_time': 20.576,
                           'text': 'performance and reliability of GIT lab.com '
                                   'as we look'},
              'score': 0.566716492,
              'values': []},
             {'id': '203',
              'metadata': {'meeting_id': 3.0,
                           'speaker': 'spk_3',
                           'start_time': 10.2445,
                           'text': 'SEOP experience where we have several '
        

#### Open AI

In [34]:
query  = "What is the significance of GLAB pages supporting multiple versions, as mentioned in the meeting"
downstr_response = index.query(
    vector= embed.embed_documents([query])[0],
    # filter={
    #     "meeting_id": {"$in":[1, 2]}
    # },
    namespace=openai_namespace, 
    top_k=20,
    include_metadata=True,
)
downstr_response

{'matches': [{'id': '26',
              'metadata': {'meeting_id': 1.0,
                           'speaker': 'spk_1',
                           'start_time': 2.4546,
                           'text': 'today. GLAB pages supports only a single '
                                   'version of a paid site um As a result,'},
              'score': 0.825050175,
              'values': []},
             {'id': '159',
              'metadata': {'meeting_id': 2.0,
                           'speaker': 'spk_5',
                           'start_time': 16.2496,
                           'text': 'We are working towards bringing Glab Duo, '
                                   'which is our chat service uh towards self '
                                   'managed like we did with code '
                                   'suggestions.'},
              'score': 0.797058582,
              'values': []},
             {'id': '136',
              'metadata': {'meeting_id': 2.0,
                      

#### Open AI

In [177]:
query  = "What AI experiment does Sam White discuss in the SEC section for resolving vulnerabilities?"
downstr_response = index.query(
    vector= embed.embed_documents([query])[0],
    # filter={
    #     "meeting_id": {"$in":[1, 2]}
    # },
    namespace=openai_namespace, 
    top_k=20,
    include_metadata=True,
)
downstr_response

{'matches': [],
 'namespace': 'monthlyreleasekickoff',
 'usage': {'read_units': 0}}

##### Cohere

In [167]:
query  = "What AI experiment does Sam White discuss in the SEC section for resolving vulnerabilities?"
downstr_response = index.query(
    vector= embed_text([query])[0],
    # filter={
    #     "meeting_id": {"$in":[1, 2]}
    # },
    namespace=openai_namespace, 
    top_k=10,
    include_metadata=True,
)
downstr_response

{'matches': [{'id': '87',
              'metadata': {'meeting_id': 1.0,
                           'speaker': 'spk_1',
                           'start_time': 4.216,
                           'text': 'Um And next up, I will pass it on to Sam '
                                   'White.'},
              'score': 0.475003749,
              'values': []},
             {'id': '9',
              'metadata': {'meeting_id': 1.0,
                           'speaker': 'spk_0',
                           'start_time': 0.4267,
                           'text': 'talk about some of the improvements coming '
                                   'in sec.'},
              'score': 0.471394181,
              'values': []},
             {'id': '94',
              'metadata': {'meeting_id': 2.0,
                           'speaker': 'spk_2',
                           'start_time': 4.7098,
                           'text': 'A I in order to resolve that '
                                   'vulnerabilit